## A simple multinomial logit model using the Swissmetro dataset

In [ ]:
import sys
sys.path.append('..')

In [ ]:
# import packages
import pandas as pd

from pycmtensor import train
from pycmtensor import Dataset
from pycmtensor.expressions import Beta
import pycmtensor.scheduler as sched
import pycmtensor.optimizers as optim
from pycmtensor.models import MNL

import pycmtensor
pycmtensor.about()

In [ ]:
# read the data and load it into a dataset
df = pd.read_csv('http://transp-or.epfl.ch/data/swissmetro.dat', sep='\t')

# remove unknown choices
df = df.loc[df["CHOICE"] > 0]

# Load the DataFrame into a Dataset object
ds = Dataset(df, choice="CHOICE")
ds.scale_variable("TRAIN_TT", 100)
ds.scale_variable("SM_TT", 100)
ds.scale_variable("CAR_TT", 100)
ds.scale_variable("TRAIN_CO", 100)
ds.scale_variable("SM_CO", 100)
ds.scale_variable("CAR_CO", 100)
ds.split(frac=0.80)

# Define the alternative specific constants (ASCs) for each mode of transport
ASC_TRAIN = Beta("ASC_TRAIN", 0., None, None, 0)
ASC_SM = Beta("ASC_SM", 0., None, None, 0)
ASC_CAR = Beta("ASC_CAR", 0., None, None, 1)

# Define the coefficients for the variables
B_COST = Beta("B_COST", 0., None, None, 0)
B_TIME_TRAIN = Beta("B_TIME_TRAIN", 0., None, None, 0)
B_TIME_SM = Beta("B_TIME_SM", 0., None, None, 0)
B_TIME_CAR = Beta("B_TIME_CAR", 0., None, None, 0)
B_SEAT = Beta("B_SEAT", 0., None, None, 0)

# Define the utility functions for each mode of transport
V_TRAIN = (
	ASC_TRAIN 
	+ B_TIME_TRAIN * ds["TRAIN_TT"] 
	+ B_COST * ds["TRAIN_CO"]
)
V_SM = (
	ASC_SM 
	+ B_TIME_SM * ds["SM_TT"] 
	+ B_COST * ds["SM_CO"] 
	+ B_SEAT * ds["SM_SEATS"]
)
V_CAR = (
	ASC_CAR 
	+ B_TIME_CAR * ds["CAR_TT"] 
	+ B_COST * ds["CAR_CO"]
)

# Define the model
U = [V_TRAIN, V_SM, V_CAR]
AV = [ds["TRAIN_AV"], ds["SM_AV"], ds["CAR_AV"]]
model = MNL(ds, locals(), U, AV)

Estimate the model

In [ ]:
# Train model
model.reset_values()
train(model, ds, optimizer=optim.Adam, lr_scheduler=sched.ConstantLR, batch_size=0, max_epochs=2000, base_learning_rate=0.01, convergence_threshold=1e-3, acceptance_method=0)

Display model results:

In [ ]:
display(model.results.beta_statistics())
display(model.results.model_statistics())
display(model.results.benchmark())

Display model correlation matrices

In [ ]:
display(model.results.model_correlation_matrix())
# display(model.results.model_robust_correlation_matrix())

Plot model Loglikelihood, training and validation error

In [ ]:
model.results.show_training_plot()

Model validation prediction

In [ ]:
# predict CHOICE on the validation set and display the results.
# First three columns are the probabilities, the fourth column 
# is the predicted choice, fifth column is the actual choice.
pd.DataFrame(model.predict(ds))

Compute elasticities of SM_CO with respect to CHOICE:SM

In [ ]:
sm_co_wrt_sm = model.elasticities(ds, 1)["SM_CO"]
print(sm_co_wrt_sm.mean())

Plot the elasticities

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style("ticks")

fig, ax = plt.subplots()
sns.histplot(sm_co_wrt_sm, kde=False, ax=ax, bins=40)

sns.despine()
plt.show()